In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pvlib.solarposition import get_solarposition
from datetime import datetime, timedelta
import pandas as pd
from ipywidgets import interact, IntSlider, FloatSlider, DatePicker

def calculate_shadow_length(height, solar_elevation):
    """Calculate shadow length from object height and solar elevation angle."""
    if solar_elevation <= 0:  # No shadow at night
        return 0
    return height / np.tan(np.radians(solar_elevation))

def calculate_shadow_direction(solar_azimuth):
    """Calculate shadow direction (opposite to solar azimuth)."""
    return (solar_azimuth + 180) % 360

def daily_shade_pattern_pvlib(height, latitude, longitude, date, min_elevation=0):
    """
    Generate daily shade pattern using pvlib for solar calculations,
    limiting shadow casting to when solar elevation >= min_elevation.
    """
    # Create a timestamp series for the entire day
    times = pd.date_range(start=datetime.combine(date, datetime.min.time()),
                          end=datetime.combine(date, datetime.max.time()),
                          freq="15min", tz="UTC")
    
    # Calculate solar positions
    solar_pos = get_solarposition(times, latitude, longitude)
    
    shadow_lengths = []
    shadow_directions = []
    
    for _, row in solar_pos.iterrows():
        solar_elev = row['elevation']
        solar_azim = row['azimuth']
        
        if solar_elev >= min_elevation:
            shadow_length = calculate_shadow_length(height, solar_elev)
            shadow_direction = calculate_shadow_direction(solar_azim)
        else:
            shadow_length = 0
            shadow_direction = None  # No shadow
        
        shadow_lengths.append(shadow_length)
        shadow_directions.append(shadow_direction)
    
    return times, shadow_lengths, shadow_directions

def plot_shade_pattern(times, shadow_lengths, shadow_directions, date):
    """Visualize the shade pattern with North oriented up."""
    plt.figure(figsize=(20, 12))
    
    for length, direction in zip(shadow_lengths, shadow_directions):
        if length > 0 and direction is not None:
            # Convert polar to Cartesian coordinates (North-up orientation)
            x = np.sin(np.radians(direction)) * length
            y = np.cos(np.radians(direction)) * length
            plt.plot([0, x], [0, y], color='blue', alpha=0.3)
    
    # Plot the object
    plt.scatter(0, 0, color='black', label='Object', zorder=5)
    plt.xlabel("East-West Direction (meters)")
    plt.ylabel("North-South Direction (meters)")
    plt.title(f"Shade Pattern (North Up) on {date.strftime('%Y-%m-%d')}")
    plt.legend()
    plt.grid()
    plt.axis('equal')
    plt.show()

# Interactive function
def interactive_shade(height, latitude, longitude, date, min_elevation):
    times, shadow_lengths, shadow_directions = daily_shade_pattern_pvlib(
        height, latitude, longitude, date, min_elevation
    )
    plot_shade_pattern(times, shadow_lengths, shadow_directions, date)

# Widgets
interact(
    interactive_shade,
    height=FloatSlider(value=5, min=1, max=20, step=0.5, description='Height (m)'),
    latitude=FloatSlider(value=52.0, min=-90, max=90, step=0.1, description='Latitude'),
    longitude=FloatSlider(value=4.9, min=-180, max=180, step=0.1, description='Longitude'),
    date=DatePicker(value=datetime(2024, 6, 21), description='Date'),
    min_elevation=FloatSlider(value=0, min=-10, max=30, step=0.5, description='Min Elevation (°)')
);

interactive(children=(FloatSlider(value=5.0, description='Height (m)', max=20.0, min=1.0, step=0.5), FloatSlid…

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pvlib.solarposition import get_solarposition
from datetime import datetime, timedelta
import pandas as pd
from ipywidgets import interact, FloatSlider, DatePicker, Dropdown

def calculate_shadow_length(height, solar_elevation):
    """Calculate shadow length from object height and solar elevation angle."""
    if solar_elevation <= 0:  # No shadow at night
        return 0
    return height / np.tan(np.radians(solar_elevation))

def calculate_shadow_direction(solar_azimuth):
    """Calculate shadow direction (opposite to solar azimuth)."""
    return (solar_azimuth + 180) % 360

def daily_shade_pattern_pvlib(height, canopy_radius, object_type, latitude, longitude, date, min_elevation=0):
    """
    Generate daily shade pattern for different object types using pvlib for solar calculations.
    """
    # Create a timestamp series for the entire day
    times = pd.date_range(start=datetime.combine(date, datetime.min.time()),
                          end=datetime.combine(date, datetime.max.time()),
                          freq="15min", tz="UTC")
    
    # Calculate solar positions
    solar_pos = get_solarposition(times, latitude, longitude)
    
    shadow_lengths = []
    shadow_directions = []
    canopy_shadows = []  # Additional shadows for trees
    
    for _, row in solar_pos.iterrows():
        solar_elev = row['elevation']
        solar_azim = row['azimuth']
        
        if solar_elev >= min_elevation:
            shadow_length = calculate_shadow_length(height, solar_elev)
            shadow_direction = calculate_shadow_direction(solar_azim)
            
            # Additional shadowing for trees
            if object_type == "Tree" and canopy_radius > 0:
                canopy_shadow_length = calculate_shadow_length(height + canopy_radius, solar_elev)
            else:
                canopy_shadow_length = 0
        else:
            shadow_length = 0
            shadow_direction = None  # No shadow
            canopy_shadow_length = 0
        
        shadow_lengths.append(shadow_length)
        shadow_directions.append(shadow_direction)
        canopy_shadows.append(canopy_shadow_length)
    
    return times, shadow_lengths, shadow_directions, canopy_shadows

def plot_shade_pattern(times, shadow_lengths, shadow_directions, canopy_shadows, object_type, canopy_radius, date):
    """Visualize the shade pattern for different object types with North oriented up."""
    plt.figure(figsize=(20, 12))
    
    for length, direction, canopy_length in zip(shadow_lengths, shadow_directions, canopy_shadows):
        if length > 0 and direction is not None:
            # Convert polar to Cartesian coordinates (North-up orientation)
            x = np.sin(np.radians(direction)) * length
            y = np.cos(np.radians(direction)) * length
            plt.plot([0, x], [0, y], color='blue', alpha=0.3)
            
            # Plot additional shadow for tree canopies
            if object_type == "Tree" and canopy_length > 0:
                canopy_x = np.sin(np.radians(direction)) * canopy_length
                canopy_y = np.cos(np.radians(direction)) * canopy_length
                plt.plot([0, canopy_x], [0, canopy_y], color='green', alpha=0.2)
    
    # Plot the object
    plt.scatter(0, 0, color='black', label='Object', zorder=5)
    if object_type == "Tree" and canopy_radius > 0:
        canopy_circle = plt.Circle((0, 0), canopy_radius, color='green', alpha=0.3, label='Canopy')
        plt.gca().add_artist(canopy_circle)
    plt.xlabel("East-West Direction (meters)")
    plt.ylabel("North-South Direction (meters)")
    plt.title(f"{object_type} Shade Pattern (North Up) on {date.strftime('%Y-%m-%d')}")
    plt.legend()
    plt.grid()
    plt.axis('equal')
    plt.show()

# Interactive function
def interactive_shade(object_type, height, canopy_radius, latitude, longitude, date, min_elevation):
    times, shadow_lengths, shadow_directions, canopy_shadows = daily_shade_pattern_pvlib(
        height, canopy_radius, object_type, latitude, longitude, date, min_elevation
    )
    plot_shade_pattern(times, shadow_lengths, shadow_directions, canopy_shadows, object_type, canopy_radius, date)

# Widgets
interact(
    interactive_shade,
    object_type=Dropdown(options=["Building", "Tree"], value="Building", description="Object Type"),
    height=FloatSlider(value=5, min=1, max=20, step=0.5, description='Height (m)'),
    canopy_radius=FloatSlider(value=2, min=0, max=10, step=0.5, description='Canopy Radius (m)'),
    latitude=FloatSlider(value=52.0, min=-90, max=90, step=0.1, description='Latitude'),
    longitude=FloatSlider(value=4.9, min=-180, max=180, step=0.1, description='Longitude'),
    date=DatePicker(value=datetime(2024, 6, 21), description='Date'),
    min_elevation=FloatSlider(value=0, min=-10, max=30, step=0.5, description='Min Elevation (°)')
);

interactive(children=(Dropdown(description='Object Type', options=('Building', 'Tree'), value='Building'), Flo…

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from pvlib.solarposition import get_solarposition
from datetime import datetime, timedelta
import pandas as pd
from ipywidgets import interact, FloatSlider, IntSlider, DatePicker, Dropdown

def calculate_shadow_length(height, solar_elevation):
    """Calculate shadow length from object height and solar elevation angle."""
    if solar_elevation <= 0:  # No shadow at night
        return 0
    return height / np.tan(np.radians(solar_elevation))

def calculate_shadow_direction(solar_azimuth):
    """Calculate shadow direction (opposite to solar azimuth)."""
    return (solar_azimuth + 180) % 360

def daily_shade_pattern_pvlib(height, canopy_radius, object_type, latitude, longitude, date, min_elevation=0):
    """
    Generate daily shade pattern for different object types using pvlib for solar calculations.
    """
    # Create a timestamp series for the entire day
    times = pd.date_range(start=datetime.combine(date, datetime.min.time()),
                          end=datetime.combine(date, datetime.max.time()),
                          freq="15min", tz="UTC")
    
    # Calculate solar positions
    solar_pos = get_solarposition(times, latitude, longitude)
    
    shadow_lengths = []
    shadow_directions = []
    canopy_shadows = []  # Additional shadows for trees
    
    for _, row in solar_pos.iterrows():
        solar_elev = row['elevation']
        solar_azim = row['azimuth']
        
        if solar_elev >= min_elevation:
            shadow_length = calculate_shadow_length(height, solar_elev)
            shadow_direction = calculate_shadow_direction(solar_azim)
            
            # Additional shadowing for trees
            if object_type == "Tree" and canopy_radius > 0:
                canopy_shadow_length = calculate_shadow_length(height + canopy_radius, solar_elev)
            else:
                canopy_shadow_length = 0
        else:
            shadow_length = 0
            shadow_direction = None  # No shadow
            canopy_shadow_length = 0
        
        shadow_lengths.append(shadow_length)
        shadow_directions.append(shadow_direction)
        canopy_shadows.append(canopy_shadow_length)
    
    return times, shadow_lengths, shadow_directions, canopy_shadows

def plot_shade_pattern(times, shadow_lengths, shadow_directions, canopy_shadows, object_type, canopy_radius, height, width, date):
    """Visualize the shade pattern with true geometries and colored rays."""
    # Colormap for time of day
    norm = Normalize(vmin=0, vmax=len(times))
    cmap = plt.get_cmap('plasma')
    
    plt.figure(figsize=(20, 12))
    ax = plt.gca()  # Get the current axes
    
    for i, (length, direction, canopy_length) in enumerate(zip(shadow_lengths, shadow_directions, canopy_shadows)):
        if length > 0 and direction is not None:
            color = cmap(norm(i))  # Color based on time
            
            # Convert polar to Cartesian coordinates (North-up orientation)
            x = np.sin(np.radians(direction)) * length
            y = np.cos(np.radians(direction)) * length
            plt.plot([0, x], [0, y], color=color, alpha=0.5)
            
            # Plot additional shadow for tree canopies
            if object_type == "Tree" and canopy_length > 0:
                canopy_x = np.sin(np.radians(direction)) * canopy_length
                canopy_y = np.cos(np.radians(direction)) * canopy_length
                plt.plot([0, canopy_x], [0, canopy_y], color=color, alpha=0.2)
    
    # Plot the object
    if object_type == "Building":
        rect = plt.Rectangle((-width / 2, 0), width, -height, color='gray', alpha=0.6, label='Building')
        ax.add_artist(rect)
    elif object_type == "Tree" and canopy_radius > 0:
        canopy_circle = plt.Circle((0, 0), canopy_radius, color='green', alpha=0.3, label='Canopy')
        ax.add_artist(canopy_circle)
    
    # Add colormap legend
    sm = ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])  # Set an array for the ScalarMappable
    cbar = plt.colorbar(sm, ax=ax, orientation='horizontal', pad=0.1)
    cbar.set_label('Time of Day')
    
    plt.scatter(0, 0, color='black', label='Object', zorder=5)
    plt.xlabel("East-West Direction (meters)")
    plt.ylabel("North-South Direction (meters)")
    plt.title(f"{object_type} Shade Pattern (North Up) on {date.strftime('%Y-%m-%d')}")
    plt.legend()
    plt.grid()
    plt.axis('equal')
    plt.show()

# Interactive function
def interactive_shade(object_type, height, canopy_radius, width, latitude, longitude, date, min_elevation):
    times, shadow_lengths, shadow_directions, canopy_shadows = daily_shade_pattern_pvlib(
        height, canopy_radius, object_type, latitude, longitude, date, min_elevation
    )
    plot_shade_pattern(times, shadow_lengths, shadow_directions, canopy_shadows, object_type, canopy_radius, height, width, date)

# Widgets
interact(
    interactive_shade,
    object_type=Dropdown(options=["Building", "Tree"], value="Building", description="Object Type"),
    height=FloatSlider(value=5, min=1, max=20, step=0.5, description='Height (m)'),
    canopy_radius=FloatSlider(value=2, min=0, max=10, step=0.5, description='Canopy Radius (m)'),
    width=FloatSlider(value=5, min=1, max=20, step=0.5, description='Width (m)'),
    latitude=FloatSlider(value=52.0, min=-90, max=90, step=0.1, description='Latitude'),
    longitude=FloatSlider(value=4.9, min=-180, max=180, step=0.1, description='Longitude'),
    date=DatePicker(value=datetime(2024, 6, 21), description='Date'),
    min_elevation=FloatSlider(value=0, min=-10, max=30, step=0.5, description='Min Elevation (°)')
);

interactive(children=(Dropdown(description='Object Type', options=('Building', 'Tree'), value='Building'), Flo…

In [11]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from pvlib.solarposition import get_solarposition
from datetime import datetime, timedelta
import pandas as pd
from ipywidgets import interact, FloatSlider, IntSlider, DatePicker, Dropdown

def calculate_shadow_length(height, solar_elevation):
    """Calculate shadow length from object height and solar elevation angle."""
    if solar_elevation <= 0:  # No shadow at night
        return 0
    return height / np.tan(np.radians(solar_elevation))

def calculate_shadow_direction(solar_azimuth):
    """Calculate shadow direction (opposite to solar azimuth)."""
    return (solar_azimuth + 180) % 360

def daily_shade_pattern_pvlib(height, canopy_radius, object_type, latitude, longitude, date, min_elevation=0):
    """
    Generate daily shade pattern for different object types using pvlib for solar calculations.
    """
    # Create a timestamp series for the entire day
    times = pd.date_range(start=datetime.combine(date, datetime.min.time()),
                          end=datetime.combine(date, datetime.max.time()),
                          freq="15min", tz="UTC")
    
    # Calculate solar positions
    solar_pos = get_solarposition(times, latitude, longitude)
    
    shadow_lengths = []
    shadow_directions = []
    canopy_shadows = []  # Additional shadows for trees
    
    for _, row in solar_pos.iterrows():
        solar_elev = row['elevation']
        solar_azim = row['azimuth']
        
        if solar_elev >= min_elevation:
            shadow_length = calculate_shadow_length(height, solar_elev)
            shadow_direction = calculate_shadow_direction(solar_azim)
            
            # Additional shadowing for trees
            if object_type == "Tree" and canopy_radius > 0:
                canopy_shadow_length = calculate_shadow_length(height + canopy_radius, solar_elev)
            else:
                canopy_shadow_length = 0
        else:
            shadow_length = 0
            shadow_direction = None  # No shadow
            canopy_shadow_length = 0
        
        shadow_lengths.append(shadow_length)
        shadow_directions.append(shadow_direction)
        canopy_shadows.append(canopy_shadow_length)
    
    return times, shadow_lengths, shadow_directions, canopy_shadows

def plot_shade_pattern(times, shadow_lengths, shadow_directions, canopy_shadows, object_type, canopy_radius, height, width, date):
    """Visualize the shade pattern with realistic shadow casting."""
    # Colormap for time of day
    norm = Normalize(vmin=0, vmax=len(times))
    cmap = plt.get_cmap('plasma')
    
    plt.figure(figsize=(20, 12))
    ax = plt.gca()  # Get the current axes
    
    for i, (length, direction, canopy_length) in enumerate(zip(shadow_lengths, shadow_directions, canopy_shadows)):
        if length > 0 and direction is not None:
            color = cmap(norm(i))  # Color based on time
            
            # Shadow direction vector
            dx = np.sin(np.radians(direction))
            dy = np.cos(np.radians(direction))
            
            if object_type == "Building":
                # Define building corners (bottom-left, bottom-right, top-right, top-left)
                corners = [
                    (-width / 2, 0),  # Bottom-left corner
                    (width / 2, 0),   # Bottom-right corner
                    (-width / 2, -height),  # Top-left corner
                    (width / 2, -height)    # Top-right corner
                ]
                
                # Determine edges facing away from the sun
                for (x1, y1), (x2, y2) in zip(corners, corners[1:] + corners[:1]):
                    # Calculate edge direction vector
                    edge_dx = x2 - x1
                    edge_dy = y2 - y1
                    
                    # Calculate edge normal vector
                    nx = -edge_dy
                    ny = edge_dx
                    
                    # Dot product to check if edge is shaded
                    if dx * nx + dy * ny > 0:  # Edge facing away from the sun
                        shadow_x1 = x1 + dx * length
                        shadow_y1 = y1 + dy * length
                        shadow_x2 = x2 + dx * length
                        shadow_y2 = y2 + dy * length
                        plt.plot([x1, shadow_x1], [y1, shadow_y1], color=color, alpha=0.5)
                        plt.plot([x2, shadow_x2], [y2, shadow_y2], color=color, alpha=0.5)
            elif object_type == "Tree":
                # Cast shadows from the canopy boundary facing away from the sun
                ray_number = int(canopy_radius*10)
                canopy_angles = np.linspace(0, 360, num=ray_number)
                for angle in canopy_angles:
                    canopy_dx = canopy_radius * np.cos(np.radians(angle))
                    canopy_dy = canopy_radius * np.sin(np.radians(angle))
                    
                    # Vector from canopy point to shadow direction
                    dot_product = dx * canopy_dx + dy * canopy_dy
                    if dot_product > 0:  # Canopy point facing away from the sun
                        shadow_x = canopy_dx + dx * length
                        shadow_y = canopy_dy + dy * length
                        plt.plot([canopy_dx, shadow_x], [canopy_dy, shadow_y], color=color, alpha=0.2)
    
    # Plot the object
    if object_type == "Building":
        rect = plt.Rectangle((-width / 2, -height), width, height, color='gray', alpha=0.6, label='Building')
        ax.add_artist(rect)
    elif object_type == "Tree" and canopy_radius > 0:
        canopy_circle = plt.Circle((0, 0), canopy_radius, color='green', alpha=0.3, label='Canopy')
        ax.add_artist(canopy_circle)
    
    # Add colormap legend
    sm = ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])  # Set an array for the ScalarMappable
    cbar = plt.colorbar(sm, ax=ax, orientation='horizontal', pad=0.1)
    cbar.set_label('Time of Day')
    
    # Replace ticks with time labels
    time_labels = [time.strftime('%H:%M') for time in times[::len(times) // 6]]
    cbar.set_ticks(np.linspace(0, len(times), len(time_labels)))
    cbar.set_ticklabels(time_labels)
    
    plt.scatter(0, 0, color='black', label='Object', zorder=5)
    plt.xlabel("East-West Direction (meters)")
    plt.ylabel("North-South Direction (meters)")
    plt.title(f"{object_type} Shade Pattern (North Up) on {date.strftime('%Y-%m-%d')}")
    plt.legend()
    plt.grid()
    plt.axis('equal')
    plt.show()

# Interactive function
def interactive_shade(object_type, height, canopy_radius, width, latitude, longitude, date, min_elevation):
    times, shadow_lengths, shadow_directions, canopy_shadows = daily_shade_pattern_pvlib(
        height, canopy_radius, object_type, latitude, longitude, date, min_elevation
    )
    plot_shade_pattern(times, shadow_lengths, shadow_directions, canopy_shadows, object_type, canopy_radius, height, width, date)

# Widgets
interact(
    interactive_shade,
    object_type=Dropdown(options=["Building", "Tree"], value="Building", description="Object Type"),
    height=FloatSlider(value=5, min=1, max=20, step=0.5, description='Height (m)'),
    canopy_radius=FloatSlider(value=2, min=0, max=10, step=0.5, description='Canopy Radius (m)'),
    width=FloatSlider(value=5, min=1, max=20, step=0.5, description='Width (m)'),
    latitude=FloatSlider(value=52.0, min=-90, max=90, step=0.1, description='Latitude'),
    longitude=FloatSlider(value=4.9, min=-180, max=180, step=0.1, description='Longitude'),
    date=DatePicker(value=datetime(2024, 6, 21), description='Date'),
    min_elevation=FloatSlider(value=0, min=-10, max=30, step=0.5, description='Min Elevation (°)')
);

interactive(children=(Dropdown(description='Object Type', options=('Building', 'Tree'), value='Building'), Flo…

### Highlight individual timesteps:

In [ ]:
def plot_shade_pattern(times, shadow_lengths, shadow_directions, canopy_shadows, object_type, canopy_radius, height, width, date, highlight_timestep=None):
    """Visualize the shade pattern with all timesteps and highlight a specific timestep."""
    # Colormap for time of day
    norm = Normalize(vmin=0, vmax=len(times))
    cmap = plt.get_cmap('plasma')
    
    plt.figure(figsize=(20, 12))
    ax = plt.gca()  # Get the current axes
    
    for i, (length, direction, canopy_length) in enumerate(zip(shadow_lengths, shadow_directions, canopy_shadows)):
        if length > 0 and direction is not None:
            # Use a faint color for non-highlighted rays
            alpha = 0.3 if highlight_timestep is None or i != highlight_timestep else 1.0
            linewidth = 1 if highlight_timestep is None or i != highlight_timestep else 2
            color = cmap(norm(i))
            
            # Shadow direction vector
            dx = np.sin(np.radians(direction))
            dy = np.cos(np.radians(direction))
            
            if object_type == "Building":
                # Define building corners (bottom-left, bottom-right, top-right, top-left)
                corners = [
                    (-width / 2, 0),  # Bottom-left corner
                    (width / 2, 0),   # Bottom-right corner
                    (width / 2, -height),  # Top-right corner
                    (-width / 2, -height)  # Top-left corner
                ]
                
                # Cast shadows from edges facing away from the sun
                for (x1, y1), (x2, y2) in zip(corners, corners[1:] + corners[:1]):
                    # Calculate edge normal vector
                    nx = y2 - y1
                    ny = -(x2 - x1)
                    
                    # Dot product to check if edge is shaded
                    if dx * nx + dy * ny > 0:  # Edge facing away from the sun
                        shadow_x1 = x1 + dx * length
                        shadow_y1 = y1 + dy * length
                        shadow_x2 = x2 + dx * length
                        shadow_y2 = y2 + dy * length
                        plt.plot([x1, shadow_x1], [y1, shadow_y1], color=color, alpha=alpha, linewidth=linewidth)
                        plt.plot([x2, shadow_x2], [y2, shadow_y2], color=color, alpha=alpha, linewidth=linewidth)
            elif object_type == "Tree":
                # Cast shadows from the canopy boundary facing away from the sun
                canopy_angles = np.linspace(0, 360, num=int(canopy_radius * 10))
                for angle in canopy_angles:
                    canopy_dx = canopy_radius * np.cos(np.radians(angle))
                    canopy_dy = canopy_radius * np.sin(np.radians(angle))
                    
                    # Vector from canopy point to shadow direction
                    dot_product = dx * canopy_dx + dy * canopy_dy
                    if dot_product > 0:  # Canopy point facing away from the sun
                        shadow_x = canopy_dx + dx * length
                        shadow_y = canopy_dy + dy * length
                        plt.plot([canopy_dx, shadow_x], [canopy_dy, shadow_y], color=color, alpha=alpha, linewidth=linewidth)
    
    # Highlight current timestep time
    if highlight_timestep is not None:
        current_time = times[highlight_timestep].strftime('%H:%M')
        plt.text(0.05, 0.95, f"Time: {current_time}", transform=ax.transAxes,
                 fontsize=16, verticalalignment='top', color='black', bbox=dict(facecolor='white', alpha=0.8))
    
    # Plot the object
    if object_type == "Building":
        rect = plt.Rectangle((-width / 2, -height), width, height, color='gray', alpha=0.6, label='Building')
        ax.add_artist(rect)
    elif object_type == "Tree" and canopy_radius > 0:
        canopy_circle = plt.Circle((0, 0), canopy_radius, color='green', alpha=0.3, label='Canopy')
        ax.add_artist(canopy_circle)
    
    # Add colormap legend
    sm = ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])  # Set an array for the ScalarMappable
    cbar = plt.colorbar(sm, ax=ax, orientation='horizontal', pad=0.1)
    cbar.set_label('Time of Day')
    
    # Replace ticks with time labels
    time_labels = [time.strftime('%H:%M') for time in times[::len(times) // 6]]
    cbar.set_ticks(np.linspace(0, len(times), len(time_labels)))
    cbar.set_ticklabels(time_labels)
    
    plt.scatter(0, 0, color='none', label='Object', zorder=5)
    plt.xlabel("East-West Direction (meters)")
    plt.ylabel("North-South Direction (meters)")
    plt.title(f"{object_type} Shade Pattern (North Up) on {date.strftime('%Y-%m-%d')}")
    # plt.legend()
    plt.grid()
    plt.axis('equal')
    plt.show()


# Interactive function
def interactive_shade(object_type, height, canopy_radius, width, latitude, longitude, date, min_elevation, highlight_timestep):
    times, shadow_lengths, shadow_directions, canopy_shadows = daily_shade_pattern_pvlib(
        height, canopy_radius, object_type, latitude, longitude, date, min_elevation
    )
    plot_shade_pattern(times, shadow_lengths, shadow_directions, canopy_shadows, object_type, canopy_radius, height, width, date, highlight_timestep=highlight_timestep)


# Widgets
interact(
    interactive_shade,
    object_type=Dropdown(options=["Building", "Tree"], value="Building", description="Object Type"),
    height=FloatSlider(value=5, min=1, max=20, step=0.5, description='Height (m)'),
    canopy_radius=FloatSlider(value=2, min=0, max=10, step=0.5, description='Canopy Radius (m)'),
    width=FloatSlider(value=5, min=1, max=20, step=0.5, description='Width (m)'),
    latitude=FloatSlider(value=52.0, min=-90, max=90, step=0.1, description='Latitude'),
    longitude=FloatSlider(value=4.9, min=-180, max=180, step=0.1, description='Longitude'),
    date=DatePicker(value=datetime(2024, 6, 21), description='Date'),
    min_elevation=FloatSlider(value=0, min=-10, max=30, step=0.5, description='Min Elevation (°)'),
    highlight_timestep=IntSlider(value=None, min=0, max=96, step=1, description='Highlight Timestep')
);

interactive(children=(Dropdown(description='Object Type', options=('Building', 'Tree'), value='Building'), Flo…

### Starting with multiple trees

In [10]:
def plot_shade_pattern_multi_trees(times, shadow_lengths, shadow_directions, tree_positions, tree_canopy_radii, tree_heights, date, highlight_timestep=None):
    """Visualize the shade pattern with multiple trees and highlight a specific timestep."""
    # Colormap for time of day
    norm = Normalize(vmin=0, vmax=len(times))
    cmap = plt.get_cmap('plasma')
    
    plt.figure(figsize=(20, 12))
    ax = plt.gca()  # Get the current axes
    
    for i, (length, direction) in enumerate(zip(shadow_lengths, shadow_directions)):
        if length > 0 and direction is not None:
            # Use a faint color for non-highlighted rays
            alpha = 0.3 if highlight_timestep is None or i != highlight_timestep else 1.0
            linewidth = 1 if highlight_timestep is None or i != highlight_timestep else 2
            color = cmap(norm(i))
            
            # Shadow direction vector
            dx = np.sin(np.radians(direction))
            dy = np.cos(np.radians(direction))
            
            for (x_pos, y_pos), canopy_radius, height in zip(tree_positions, tree_canopy_radii, tree_heights):
                # Cast shadows from the canopy boundary facing away from the sun
                canopy_angles = np.linspace(0, 360, num=int(canopy_radius * 10))
                for angle in canopy_angles:
                    canopy_dx = canopy_radius * np.cos(np.radians(angle))
                    canopy_dy = canopy_radius * np.sin(np.radians(angle))
                    
                    # Adjust for tree position
                    tree_canopy_x = x_pos + canopy_dx
                    tree_canopy_y = y_pos + canopy_dy
                    
                    # Vector from canopy point to shadow direction
                    dot_product = dx * canopy_dx + dy * canopy_dy
                    if dot_product > 0:  # Canopy point facing away from the sun
                        shadow_x = tree_canopy_x + dx * length
                        shadow_y = tree_canopy_y + dy * length
                        plt.plot([tree_canopy_x, shadow_x], [tree_canopy_y, shadow_y], color=color, alpha=alpha, linewidth=linewidth)
    
    # Highlight current timestep time
    if highlight_timestep is not None:
        current_time = times[highlight_timestep].strftime('%H:%M')
        plt.text(0.05, 0.95, f"Time: {current_time}", transform=ax.transAxes,
                 fontsize=16, verticalalignment='top', color='black', bbox=dict(facecolor='white', alpha=0.8))
    
    # Plot the trees
    for (x_pos, y_pos), canopy_radius in zip(tree_positions, tree_canopy_radii):
        canopy_circle = plt.Circle((x_pos, y_pos), canopy_radius, color='green', alpha=0.3, label='Canopy')
        ax.add_artist(canopy_circle)
    
    # Add colormap legend
    sm = ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])  # Set an array for the ScalarMappable
    cbar = plt.colorbar(sm, ax=ax, orientation='horizontal', pad=0.1)
    cbar.set_label('Time of Day')
    
    # Replace ticks with time labels
    time_labels = [time.strftime('%H:%M') for time in times[::len(times) // 6]]
    cbar.set_ticks(np.linspace(0, len(times), len(time_labels)))
    cbar.set_ticklabels(time_labels)
    
    plt.scatter(0, 0, color='black', label='Object', zorder=5)
    plt.xlabel("East-West Direction (meters)")
    plt.ylabel("North-South Direction (meters)")
    plt.title(f"Shade Pattern for Multiple Trees (North Up) on {date.strftime('%Y-%m-%d')}")
    plt.legend()
    plt.grid()
    plt.axis('equal')
    plt.show()


# Interactive function
def interactive_multi_trees(tree_count, latitude, longitude, date, min_elevation, highlight_timestep):
    # Generate tree positions and attributes
    tree_positions = [(i * 10, i * 5) for i in range(tree_count)]  # Example positions
    tree_canopy_radii = [3 for _ in range(tree_count)]  # Fixed canopy radius for simplicity
    tree_heights = [5 for _ in range(tree_count)]  # Fixed height for simplicity
    
    # Calculate shadows for all timesteps
    times = pd.date_range(start=datetime.combine(date, datetime.min.time()),
                          end=datetime.combine(date, datetime.max.time()),
                          freq="15min", tz="UTC")
    solar_pos = get_solarposition(times, latitude, longitude)
    
    shadow_lengths = []
    shadow_directions = []
    for _, row in solar_pos.iterrows():
        solar_elev = row['elevation']
        solar_azim = row['azimuth']
        if solar_elev >= min_elevation:
            shadow_lengths.append(calculate_shadow_length(tree_heights[0], solar_elev))
            shadow_directions.append(calculate_shadow_direction(solar_azim))
        else:
            shadow_lengths.append(0)
            shadow_directions.append(None)
    
    # Plot the shadows
    plot_shade_pattern_multi_trees(times, shadow_lengths, shadow_directions, tree_positions, tree_canopy_radii, tree_heights, date, highlight_timestep)


# Widgets
interact(
    interactive_multi_trees,
    tree_count=IntSlider(value=3, min=1, max=10, step=1, description='Number of Trees'),
    latitude=FloatSlider(value=52.0, min=-90, max=90, step=0.1, description='Latitude'),
    longitude=FloatSlider(value=4.9, min=-180, max=180, step=0.1, description='Longitude'),
    date=DatePicker(value=datetime(2024, 6, 21), description='Date'),
    min_elevation=FloatSlider(value=10, min=-10, max=30, step=0.5, description='Min Elevation (°)'),
    highlight_timestep=IntSlider(value=None, min=0, max=96, step=1, description='Highlight Timestep')
);

interactive(children=(IntSlider(value=3, description='Number of Trees', max=10, min=1), FloatSlider(value=52.0…

### more realistic trees

In [ ]:
import random
from matplotlib.patches import Ellipse  # Import Ellipse from patches
import numpy as np
import matplotlib.pyplot as plt
from pvlib.solarposition import get_solarposition
from datetime import datetime, timedelta
import pandas as pd
from ipywidgets import interact, IntSlider, FloatSlider, DatePicker

from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

# Persistent storage for tree positions
tree_positions = []
plot_bounds = {"x_min": None, "x_max": None, "y_min": None, "y_max": None}


def generate_random_tree_positions(tree_count, tree_radius):
    """
    Generate random tree positions within a dynamically sized circular area,
    ensuring minimum spacing between trees.
    """
    global tree_positions
    scaling_factor = 2  # Adjust this factor to add more space
    placement_radius = tree_radius * (tree_count ** 0.5) * scaling_factor
    
    # Only regenerate positions if the count changes
    if len(tree_positions) != tree_count:
        positions = []
        attempts = 0  # Track attempts to avoid infinite loops
        max_attempts = 1000
        
        while len(positions) < tree_count and attempts < max_attempts:
            # Generate a random position within the placement area
            r = random.uniform(0, placement_radius)
            theta = random.uniform(0, 2 * np.pi)
            x = r * np.cos(theta)
            y = r * np.sin(theta)
            
            # Check distance to all existing trees
            valid = True
            for px, py in positions:
                distance = ((x - px) ** 2 + (y - py) ** 2) ** 0.5
                if distance < tree_radius:  # Tree radius as minimum distance
                    valid = False
                    break
            
            if valid:
                positions.append((x, y))
            
            attempts += 1
        
        if attempts == max_attempts:
            print("DEBUG: Maximum attempts reached.")
            print(f"Generated positions so far: {positions}")
            raise ValueError("Could not place all trees with the required spacing. Try reducing the tree count.")
        
        # Debug: Ensure all distances are valid
        for i, (x1, y1) in enumerate(positions):
            for j, (x2, y2) in enumerate(positions):
                if i != j:
                    distance = ((x1 - x2) ** 2 + (y1 - y2) ** 2) ** 0.5
                    if distance < tree_radius:
                        print(f"DEBUG: Overlap detected between tree {i} and tree {j}. Distance: {distance}")
                        raise ValueError("Overlap detected in generated tree positions.")
        
        tree_positions = positions
    return tree_positions


def update_plot_bounds(tree_positions, tree_canopy_radii, shadow_lengths, shadow_directions):
    """Update global plot bounds based on tree positions, canopy sizes, and shadow extents."""
    global plot_bounds
    x_min = float('inf')
    x_max = float('-inf')
    y_min = float('inf')
    y_max = float('-inf')
    
    for (x_pos, y_pos), canopy_radius in zip(tree_positions, tree_canopy_radii):
        x_min = min(x_min, x_pos - canopy_radius)
        x_max = max(x_max, x_pos + canopy_radius)
        y_min = min(y_min, y_pos - canopy_radius)
        y_max = max(y_max, y_pos + canopy_radius)
    
    # Include shadow extents
    for length, direction in zip(shadow_lengths, shadow_directions):
        if length > 0 and direction is not None:
            dx = np.sin(np.radians(direction)) * length
            dy = np.cos(np.radians(direction)) * length
            for x_pos, y_pos in tree_positions:
                x_min = min(x_min, x_pos + dx)
                x_max = max(x_max, x_pos + dx)
                y_min = min(y_min, y_pos + dy)
                y_max = max(y_max, y_pos + dy)
    
    # Add padding for better visualization
    padding = 10
    plot_bounds = {
        "x_min": x_min - padding,
        "x_max": x_max + padding,
        "y_min": y_min - padding,
        "y_max": y_max + padding,
    }


def plot_shade_pattern_multi_trees(times, shadow_lengths, shadow_directions, tree_positions, tree_canopy_radii, tree_heights, trunk_zones, date, highlight_timestep=None):
    """Visualize the shade pattern for multiple trees with ellipses and realistic shadows."""
    # Colormap for time of day
    norm = Normalize(vmin=0, vmax=len(times))
    cmap = plt.get_cmap('plasma')
    
    plt.figure(figsize=(20, 12))
    ax = plt.gca()  # Get the current axes
    
    for i, (length, direction) in enumerate(zip(shadow_lengths, shadow_directions)):
        if length > 0 and direction is not None:
            # Use a faint color for non-highlighted ellipses
            alpha = 0.15 if highlight_timestep is None or i != highlight_timestep else 1.0
            linewidth = 1 if highlight_timestep is None or i != highlight_timestep else 2
            color = cmap(norm(i))
            
            # Shadow direction vector
            dx = np.sin(np.radians(direction))
            dy = np.cos(np.radians(direction))
            
            for (x_pos, y_pos), canopy_radius, height, trunk_zone in zip(tree_positions, tree_canopy_radii, tree_heights, trunk_zones):
                # Calculate trunk height and canopy volume height
                trunk_height = height * trunk_zone
                canopy_height = height - trunk_height
                
                # Visualize the canopy shadow as an ellipse
                shadow_center_x = x_pos + dx * length / 2
                shadow_center_y = y_pos + dy * length / 2
                ellipse_width = 2 * canopy_radius
                ellipse_height = 2 * canopy_radius * (canopy_height / height)  # Scale height for canopy volume
                
                # Draw shadow ellipse
                shadow_ellipse = Ellipse((shadow_center_x, shadow_center_y), ellipse_width, ellipse_height,
                                         angle=np.degrees(np.arctan2(dy, dx)), color=color, alpha=alpha, linewidth=linewidth)
                ax.add_artist(shadow_ellipse)
    
    # Highlight current timestep time
    if highlight_timestep is not None:
        current_time = times[highlight_timestep].strftime('%H:%M')
        plt.text(0.05, 0.95, f"Time: {current_time}", transform=ax.transAxes,
                 fontsize=16, verticalalignment='top', color='black', bbox=dict(facecolor='white', alpha=0.8))
    
    # Plot the trees
    for (x_pos, y_pos), canopy_radius in zip(tree_positions, tree_canopy_radii):
        canopy_circle = plt.Circle((x_pos, y_pos), canopy_radius, color='forestgreen', alpha=0.4, label='Canopy')
        ax.add_artist(canopy_circle)

    # Add colormap legend
    sm = ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])  # Set an array for the ScalarMappable
    cbar = plt.colorbar(sm, ax=ax, orientation='horizontal', pad=0.1)
    cbar.set_label('Time of Day')
    
    # Replace ticks with time labels
    time_labels = [time.strftime('%H:%M') for time in times[::len(times) // 6]]
    cbar.set_ticks(np.linspace(0, len(times), len(time_labels)))
    cbar.set_ticklabels(time_labels)
    
    # Set plot limits using stored bounds
    global plot_bounds
    plt.xlim(plot_bounds["x_min"], plot_bounds["x_max"])
    plt.ylim(plot_bounds["y_min"], plot_bounds["y_max"])
    
    plt.xlabel("East-West Direction (meters)")
    plt.ylabel("North-South Direction (meters)")
    plt.title(f"Shade Pattern for Multiple Trees (North Up) on {date.strftime('%Y-%m-%d')}")
    # plt.legend()
    plt.grid()
    plt.axis('equal')
    plt.show()


def interactive_multi_trees(tree_count, height, radius, latitude, longitude, date, min_elevation, trunk_zone, highlight_timestep):
    global plot_bounds
    # Generate tree positions and attributes
    tree_positions = generate_random_tree_positions(tree_count, radius)
    tree_canopy_radii = [radius for _ in range(tree_count)]
    tree_heights = [height for _ in range(tree_count)]
    trunk_zones = [trunk_zone for _ in range(tree_count)]  # Uniform trunk zone percentage
    
    # Calculate shadows for all timesteps
    times = pd.date_range(start=datetime.combine(date, datetime.min.time()),
                          end=datetime.combine(date, datetime.max.time()),
                          freq="15min", tz="UTC")
    solar_pos = get_solarposition(times, latitude, longitude)
    
    shadow_lengths = []
    shadow_directions = []
    for _, row in solar_pos.iterrows():
        solar_elev = row['elevation']
        solar_azim = row['azimuth']
        if solar_elev >= min_elevation:
            shadow_lengths.append(calculate_shadow_length(tree_heights[0], solar_elev))
            shadow_directions.append(calculate_shadow_direction(solar_azim))
        else:
            shadow_lengths.append(0)
            shadow_directions.append(None)
    
    # Update bounds only when tree count changes
    if len(tree_positions) != tree_count or plot_bounds["x_min"] is None:
        update_plot_bounds(tree_positions, tree_canopy_radii, shadow_lengths, shadow_directions)
    
    # Plot the shadows
    plot_shade_pattern_multi_trees(times, shadow_lengths, shadow_directions, tree_positions, tree_canopy_radii, tree_heights, trunk_zones, date, highlight_timestep)


# Widgets
interact(
    interactive_multi_trees,
    tree_count=IntSlider(value=3, min=1, max=10, step=1, description='Number of Trees'),
    height=IntSlider(value=5, min=3, max=50, step=1, description='Tree Height (m)'),
    radius=FloatSlider(value=5, min=1, max=15, step=1, description='Tree Radius (m)'),
    latitude=FloatSlider(value=52.0, min=-90, max=90, step=0.1, description='Latitude'),
    longitude=FloatSlider(value=4.9, min=-180, max=180, step=0.1, description='Longitude'),
    date=DatePicker(value=datetime(2024, 6, 21), description='Date'),
    min_elevation=FloatSlider(value=5, min=-10, max=30, step=0.5, description='Min Elevation (°)'),
    trunk_zone=FloatSlider(value=0.3, min=0, max=1, step=0.1, description='Trunk Zone (%)'),
    highlight_timestep=IntSlider(value=None, min=0, max=96, step=1, description='Highlight Timestep')
);


interactive(children=(IntSlider(value=3, description='Number of Trees', max=10, min=1), IntSlider(value=5, des…

In [ ]:
from matplotlib.patches import RegularPolygon, Polygon
import matplotlib.pyplot as plt
import numpy as np

def plot_shade_pattern_hexagons(times, shadow_lengths, shadow_directions, tree_positions, tree_heights, hexagon_size, height_above_ground, date, highlight_timestep=None):
    """Visualize the shade pattern with hexagonal canopies and realistic shadows."""
    # Colormap for time of day
    norm = Normalize(vmin=0, vmax=len(times))
    cmap = plt.get_cmap('plasma')
    
    plt.figure(figsize=(20, 12))
    ax = plt.gca()
    
    for i, (length, direction) in enumerate(zip(shadow_lengths, shadow_directions)):
        if length > 0 and direction is not None:
            alpha = 0.15 if highlight_timestep is None or i != highlight_timestep else 1.0
            linewidth = 1 if highlight_timestep is None or i != highlight_timestep else 2
            color = cmap(norm(i))
            
            dx = np.sin(np.radians(direction))
            dy = np.cos(np.radians(direction))
            
            for x_pos, y_pos in tree_positions:
                # Hexagon shadow center
                shadow_center_x = x_pos + dx * length / 2
                shadow_center_y = y_pos + dy * length / 2
                
                # Draw shadow as a semi-transparent polygon
                shadow_corners = [
                    (x_pos + dx * length, y_pos + dy * length),
                    (x_pos - dx * length / 2, y_pos - dy * length / 2),
                    (shadow_center_x, shadow_center_y)
                ]
                shadow_polygon = Polygon(shadow_corners, closed=True, color=color, alpha=alpha)
                ax.add_patch(shadow_polygon)
    
    # Highlight current timestep time
    if highlight_timestep is not None:
        current_time = times[highlight_timestep].strftime('%H:%M')
        plt.text(0.05, 0.95, f"Time: {current_time}", transform=ax.transAxes,
                 fontsize=16, verticalalignment='top', color='black', bbox=dict(facecolor='white', alpha=0.8))
    
    # Plot the trees as hexagons
    for x_pos, y_pos in tree_positions:
        hexagon = RegularPolygon((x_pos, y_pos + height_above_ground), numVertices=6, radius=hexagon_size, 
                                 orientation=np.radians(30), color='forestgreen', alpha=0.6, edgecolor='black')
        ax.add_patch(hexagon)
    
    # Add colormap legend
    sm = ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, orientation='horizontal', pad=0.1)
    cbar.set_label('Time of Day')
    
    # Replace ticks with time labels
    time_labels = [time.strftime('%H:%M') for time in times[::len(times) // 6]]
    cbar.set_ticks(np.linspace(0, len(times), len(time_labels)))
    cbar.set_ticklabels(time_labels)
    
    # Set plot limits using stored bounds
    global plot_bounds
    plt.xlim(plot_bounds["x_min"], plot_bounds["x_max"])
    plt.ylim(plot_bounds["y_min"], plot_bounds["y_max"])
    
    plt.xlabel("East-West Direction (meters)")
    plt.ylabel("North-South Direction (meters)")
    plt.title(f"Shade Pattern with Hexagonal Trees on {date.strftime('%Y-%m-%d')}")
    plt.grid()
    plt.axis('equal')
    plt.show()

# Update the interactive function to include hexagon height
def interactive_hex_trees(tree_count, hex_size, height_above_ground, latitude, longitude, date, min_elevation, highlight_timestep):
    global plot_bounds
    # Generate tree positions
    tree_positions = generate_random_tree_positions(tree_count, hex_size)
    tree_heights = [hex_size * 2 for _ in range(tree_count)]  # Example height from size
    
    # Calculate shadows for all timesteps
    times = pd.date_range(start=datetime.combine(date, datetime.min.time()),
                          end=datetime.combine(date, datetime.max.time()),
                          freq="15min", tz="UTC")
    solar_pos = get_solarposition(times, latitude, longitude)
    
    shadow_lengths = []
    shadow_directions = []
    for _, row in solar_pos.iterrows():
        solar_elev = row['elevation']
        solar_azim = row['azimuth']
        if solar_elev >= min_elevation:
            shadow_lengths.append(calculate_shadow_length(tree_heights[0], solar_elev))
            shadow_directions.append(calculate_shadow_direction(solar_azim))
        else:
            shadow_lengths.append(0)
            shadow_directions.append(None)
    
    # Update bounds
    update_plot_bounds(tree_positions, [hex_size] * tree_count, shadow_lengths, shadow_directions)
    
    # Plot hexagonal trees
    plot_shade_pattern_hexagons(times, shadow_lengths, shadow_directions, tree_positions, tree_heights, hex_size, height_above_ground, date, highlight_timestep)

# Widgets
interact(
    interactive_hex_trees,
    tree_count=IntSlider(value=1, min=1, max=10, step=1, description='Number of Trees'),
    hex_size=FloatSlider(value=5, min=1, max=15, step=0.5, description='Hexagon Size (m)'),
    height_above_ground=FloatSlider(value=0, min=0, max=10, step=0.1, description='Height Above Ground (m)'),
    latitude=FloatSlider(value=52.0, min=-90, max=90, step=0.1, description='Latitude'),
    longitude=FloatSlider(value=4.9, min=-180, max=180, step=0.1, description='Longitude'),
    date=DatePicker(value=datetime(2024, 6, 21), description='Date'),
    min_elevation=FloatSlider(value=5, min=-10, max=30, step=0.5, description='Min Elevation (°)'),
    highlight_timestep=IntSlider(value=None, min=0, max=96, step=1, description='Highlight Timestep')
);

interactive(children=(IntSlider(value=1, description='Number of Trees', max=10, min=1), FloatSlider(value=5.0,…